In [6]:
import os
from PIL import Image

def is_valid_image(file_path):
    """
    Проверяет, является ли файл изображением и не поврежден ли он.
    """
    try:
        with Image.open(file_path) as img:
            img.verify()  # Проверка целостности изображения
        return True
    except (IOError, SyntaxError):
        return False

def clean_folder(folder_path):
    """
    Удаляет поврежденные изображения из указанной папки.
    """
    removed_files = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        # Проверяем только файлы (не папки)
        if os.path.isfile(file_path):
            if not is_valid_image(file_path):
                removed_files.append(file_path)
                os.remove(file_path)
                print(f"Удален поврежденный файл: {file_path}")
    
    if not removed_files:
        print(f"Все файлы в папке '{folder_path}' корректны.")
    else:
        print(f"Удалено {len(removed_files)} поврежденных файла(ов) из папки '{folder_path}'.")

# Указываем пути к папкам
folder1 = "D:/PetImages/train/cats"
folder2 = "D:/PetImages/train/dogs"

# Очищаем обе папки
clean_folder(folder1)
clean_folder(folder2)


Все файлы в папке 'D:/PetImages/train/cats' корректны.
Все файлы в папке 'D:/PetImages/train/dogs' корректны.


In [7]:
def preprocess_dataset(dataset):
    def process_image(image, label):
        image = tf.image.resize(image, [150, 150])  # Убедитесь, что все изображения одного размера
        return image, label

    return dataset.map(process_image)

train_dataset = preprocess_dataset(train_dataset)
val_dataset = preprocess_dataset(val_dataset)


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Загрузка данных
train_dataset = image_dataset_from_directory(
    'D:/PetImages/train',  # Путь к папке с изображениями для тренировки
    image_size=(150, 150),  # Размер изображений
    batch_size=32,  # Размер батча
    label_mode='binary',  # Бинарные метки для классификации (кошки или собаки)
    validation_split=0.2,  # Разделение на тренировочную и валидационную выборки
    subset='training',  # Для тренировочной выборки
    seed=123  # Для воспроизводимости
)

val_dataset = image_dataset_from_directory(
    'D:/PetImages/validation',  # Путь к папке с изображениями для тренировки
    image_size=(150, 150),  # Размер изображений
    batch_size=32,  # Размер батча
    label_mode='binary',  # Бинарные метки для классификации (кошки или собаки)
    validation_split=0.2,  # Разделение на тренировочную и валидационную выборки
    subset='validation',  # Для валидационной выборки
    seed=123  # Для воспроизводимости
)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 1000 files belonging to 2 classes.
Using 200 files for validation.


In [9]:
import tensorflow as tf
import os

# Параметры
image_size = (150, 150)
batch_size = 32
validation_split = 0.2

# Функция для проверки корректности изображения
def is_image(file_path):
    # Проверка расширения файла
    valid_extensions = [".jpg", ".jpeg", ".png", ".bmp"]
    return any(file_path.lower().endswith(ext) for ext in valid_extensions)

# Функция обработки изображений
def process_image(file_path, label):
    try:
        img = tf.io.read_file(file_path)  # Чтение файла
        img = tf.image.decode_image(img, channels=3)  # Универсальное декодирование изображения
        img = tf.image.resize(img, image_size)  # Изменение размера
        img = tf.cast(img, tf.float32) / 255.0  # Нормализация изображения (0-1)
        return img, label
    except Exception as e:
        print(f"Ошибка при обработке файла {file_path}: {e}")
        return None, label

# Подготовка данных
def prepare_dataset(directory):
    file_paths = []  # Список путей к изображениям
    labels = []  # Список меток

    # Поиск всех файлов и присваивание меток
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if is_image(file_path):
                file_paths.append(file_path)
                labels.append(0 if "cat" in file.lower() else 1)  # Метка: 0 для "cat", 1 для "dog"

    # Создание датасета
    file_paths_dataset = tf.data.Dataset.from_tensor_slices(file_paths)
    labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
    dataset = tf.data.Dataset.zip((file_paths_dataset, labels_dataset))

    # Применение обработки
    dataset = dataset.map(
        lambda file_path, label: process_image(file_path, label),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Удаление некорректных данных
    dataset = dataset.filter(lambda img, label: img is not None)

    return dataset

# Загрузка тренировочного и валидационного наборов
train_dataset = prepare_dataset("D:/PetImages/train")
train_size = int((1 - validation_split) * len(list(train_dataset)))

val_dataset = train_dataset.skip(train_size)
train_dataset = train_dataset.take(train_size)

# Батчи и предзагрузка
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


Ошибка при обработке файла Tensor("args_0:0", shape=(), dtype=string): 'images' contains no shape.


In [10]:
from tensorflow.keras import layers, models

# Создание модели
model = models.Sequential()

# Сверточные слои
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Полносвязные слои для классификации
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))

# Выходной слой
model.add(layers.Dense(1, activation='sigmoid'))  # Для бинарной классификации (кошки/собаки)

# Компиляция модели
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [11]:
history = model.fit(
    train_dataset,
    epochs=30,  # Количество эпох
    validation_data=val_dataset
)


Epoch 1/30


TypeError: in user code:

    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\keras\engine\input_spec.py", line 197, in assert_input_compati

In [ ]:
import matplotlib.pyplot as plt

# График точности
plt.plot(history.history['accuracy'], label='Accuracy (train)')
plt.plot(history.history['val_accuracy'], label='Accuracy (val)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over epochs')
plt.show()

# График функции потерь
plt.plot(history.history['loss'], label='Loss (train)')
plt.plot(history.history['val_loss'], label='Loss (val)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over epochs')
plt.show()
